In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import dataloader as dl

In [2]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 100
USE_CUDA = torch.cuda.is_available()
#DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)

다음 기기로 학습합니다: cuda


In [3]:
train_loader=dl.load_trainset("/home/dhk1349/Desktop/Capstone Design2/ntu/xsub/")
test_loader=dl.load_testset("/home/dhk1349/Desktop/Capstone Design2/ntu/xsub/")

In [4]:
ndf=64
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            #30*75로 해야겠음
            # input is (nc) x 30 x 75
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 15 x 37
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1,(1, 4), 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
class LSTM(nn.Module):
    #def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    def __init__(self, n_layers, hidden_dim, input_shape, n_classes, dropout_p=0.2):
        super(LSTM, self).__init__()
        print("Building Basic LSTM model...")
        self.n_layers = n_layers 
        #self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(input_shape, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        #x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        c_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.lstm(x, (h_0, c_0))  # [i, b, h]
        h_t = x[:,-1,:]
        self.dropout(h_t)
        logit = self.out(h_t)  # [b, h] -> [b, o]
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [16]:
def train(model, optimizer, train_iter):
    model.train()
    for batch in train_iter:
        x, y = np.array(batch[0][0]), batch[1]
        
        X=[]
        for i in range(30):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        X=np.array(X).reshape(1,30, 75)
        X=torch.tensor(np.array(X)).to(DEVICE)
        y=torch.tensor([y]).to(DEVICE)
        
        optimizer.zero_grad()

        logit = model(X)
        print(logit, y)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [17]:
def evaluate(model, val_iter):
    #evaluate model
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = np.array(batch[0][0]), batch[1]
        
        X=[]
        for i in range(30):
            X.append(x[np.random.randint(i*10,(1+i)*10)].reshape(-1))
        X=np.array(X).reshape(1,1,30, 75)
        X=torch.tensor(np.array(X)).unsqueeze(0).to(DEVICE)
        y=y.to(DEVICE)
        
        logit = model(X)[0][0][0]
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [7]:
model = Discriminator().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [18]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_loader)
    val_loss, val_accuracy = evaluate(model, test_loader)

    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/actionrecognition.pt')
        best_val_loss = val_loss

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
#model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_loader)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))